In [1]:

import inspect
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent
from react_agent_system_prompts import *
from members_details import members
import getpass
import os
import shutil
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override = True)



def copy_directory(src_dir):
    dst_dir= os.path.basename(src_dir)+'new'
    dst_dir = os.path.join(os.path.dirname(src_dir), dst_dir)

    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    for item in os.listdir(src_dir):
        s = os.path.join(src_dir, item)
        d = os.path.join(dst_dir, item)
        shutil.copy2(s, d)

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


from langchain_core.messages import HumanMessage
from langchain_anthropic import ChatAnthropic
from supervisor_node import make_supervisor_node
from ore_xml_agent import ore_xml_agent_node
from langgraph.graph import StateGraph, MessagesState, START
from sensitivity_agent import sensitivity_agent_node
from ore_execution_agent import ore_execution_agent_node
from ExtendedState import State


llm = ChatAnthropic(
    model="claude-3-5-haiku-latest",
    temperature=0,
    timeout=None,
    max_retries=2,
    # other params...
)

main_supervisor_node = make_supervisor_node(llm=llm, state=State, members=members)

main_agent_builder = StateGraph(State)
main_agent_builder.add_node("supervisor", main_supervisor_node)
main_agent_builder.add_node("ore_xml_agent", ore_xml_agent_node)
main_agent_builder.add_node("sensitivity_agent", sensitivity_agent_node)
main_agent_builder.add_node("ore_execution_agent", ore_execution_agent_node)

main_agent_builder.add_edge(START, "supervisor")
main_graph = main_agent_builder.compile()





ascii_representation = main_graph.get_graph().print_ascii()
print(ascii_representation)

for s in main_graph.stream(
    {"messages": [("user", "What is shock applied to EUR discounting curve. After that run the ore to compute npv results.")]}
):
    print(s)
    print("---")

In [5]:
from react_agent_system_prompts import ore_execution_agent_system_prompt_content
from config_file import file_location_prompt
from langchain_core.messages import SystemMessage, HumanMessage
from ore_xml_agent import ore_agent
from ore_execution_agent import ore_execution_agent
from ExtendedState import State
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override = True)


state = State()
state["messages"] = [HumanMessage(content="Run the ore to compute npv results.")]
input_messages = {'messages': [SystemMessage(content=ore_execution_agent_system_prompt_content),SystemMessage(content=file_location_prompt)]}
messages_list = input_messages["messages"] + state["messages"]
messages = {'messages': messages_list}
response = ore_execution_agent.invoke(messages)
state["messages"] = messages_list + response["messages"]


In [3]:
from ore_execution_tools import run_ore

test = run_ore.invoke({"file_path":"D:\Project_H\Examples\Example_1\Inputnew\ore.xml"})

'ORE run completed successfully.'

In [ ]:
test = [1,2,3,4,5]
while (len(test>))
test = test[1:]